# Import libraries

In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from dataset import NCFDataset
from recom_ncf import NCFRecommender
from evaluation import Evaluation
from helpers.index_manager import IndexManager
from helpers.dataloader_custom_functions import collate_fn
import json
import ast

# Loading data

In [2]:
df_train = pd.read_csv('../data/train-leave2.csv')
df_val = pd.read_csv('../data/val-leave2.csv')
df_test = pd.read_csv('../data/test-leave2.csv')
df_features = pd.read_csv('../data/metadata.csv')
index_manager = IndexManager()
index_manager.load('../data/index.pkl')
with open('../data/feature-dims.json') as file:
    input_dims = json.load(file)

In [3]:
features = ['tags', 'mm_total_recommendation']
feature_dims = {}
for feature in features:
    input_dim = input_dims.get(feature, 1)
    feature_dims[feature] = (input_dim, 8)

In [4]:
def str_to_list(df, cols):
    for col in cols:
        df[col] = df[col].apply(ast.literal_eval)
    return df

category_cols = ['tags']
# category_cols = ['tags', 'publishers', 'developers', 'genres', 'categories', 'supported_languages']
df_train = str_to_list(df_train, category_cols)
df_val = str_to_list(df_val, category_cols)
df_test = str_to_list(df_test, category_cols)
df_features = str_to_list(df_features, category_cols)

# Creating datasets

In [5]:
train_dataset = NCFDataset(df_train, feature_dims=feature_dims, df_features=df_features)
val_dataset = NCFDataset(df_val, feature_dims=feature_dims, df_features=df_features)

In [6]:
dataloader_params = {
    'batch_size': 2**13,
    'num_workers': 4,
    'persistent_workers': True,
    'prefetch_factor': 2,
    'pin_memory': True,
    'pin_memory_device': 'cuda',
    'collate_fn': collate_fn,
}
train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloader_params)
val_dataloader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

# Training model

In [7]:
params = {'factors': 32, 'mlp_user_item_dim': 64, 'learning_rate': 0.003, 'epochs': 1, 'optimizer': 'adagrad', 'dropout': 0.2, 'weight_decay': 0.0001, 'loss_fn': 'mse'}

unique_users = index_manager.get_indexed_values('user_id')
unique_items = index_manager.get_indexed_values('item_id')

model = NCFRecommender(unique_users, unique_items, mlp_feature_dims=feature_dims, df_features=df_features[['item_id'] + features], **params)
model.fit(train_dataloader, val_dataloader)

Epoch 1/1
Train loss: 0.173123, Validation loss: 0.153358
Training completed!


# Evaluating model

In [8]:
evaluator = Evaluation(recommender=model, test_data=df_test)
metrics = evaluator.evaluate()

Creating ground truth sets...
Generating predictions...
Processing 1 of 129461 users... (0.00%)
Memory usage: 0.213623046875 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 128 to 141
Increased item batch size from 1024 to 1126
Processing 129 of 129461 users... (0.10%)
Memory usage: 0.14697265625 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 141 to 155
Increased item batch size from 1126 to 1239
Processing 270 of 129461 users... (0.21%)
Memory usage: 0.158935546875 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 155 to 170
Increased item batch size from 1239 to 1363
Processing 425 of 129461 users... (0.33%)
Memory usage: 0.177001953125 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 170 to 187
Increased item batch size from 1363 to 1499
Processing 595 of 129461 users... (0.46%)
Memory usage: 0.19482421875 . Increasing batch size with increasing r

In [9]:
metrics

{'Hit Ratio@10': 0.03367809610616325,
 'NDCG@10': 0.017056783856614327,
 'Recall@10': 0.03367809610616325}